# ⛓️‍💥 LangChain: Models, Prompt, and Output Parsers

By the end of this notebook, you'll build a **Customer Support Classifier**
that takes raw support messages and outputs structured JSON.

**What we'll cover:**
1. Models — connecting to an LLM
2. Prompt Templates — reusable, structured prompts
3. Output Parsers — structured JSON from messy text
4. Chains (LCEL) — piping it all together

## 🛠️ Setup & Installation

In [ ]:
# Install required packages
!pip install -q langchain langchain-huggingface langchain-core

In [ ]:
import os
from google.colab import userdata

# Option 1: Use Colab Secrets (recommended)
# Go to 🔑 icon in left sidebar → Add secret: HF_API_TOKEN
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HF_API_TOKEN")

# Option 2: Paste directly (not recommended for sharing)
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_your_token_here"

print("✅ Token set!")

## 1️⃣ Your First LLM Call
Let's connect to a Hugging Face model and make our first call.
We're using the **free Inference API** — no GPU needed.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

# Load model locally on Colab GPU (free tier T4)
repo_id = "Qwen/Qwen2.5-1.5B-Instruct"

tok = AutoTokenizer.from_pretrained(repo_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    torch_dtype="auto",
    device_map="auto"
)

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True,
    return_full_text=False
)

# Wrap the pipeline for LangChain
llm = HuggingFacePipeline(pipeline=gen)

print("✅ Model loaded on", model.device)

In [ ]:
# Your first LLM call!
response = llm.invoke("Explain how coffee is made, but in 10 words and like you’re a caveman.”")
print(response)

### 🧪 YOUR TURN — Try different prompts

In [ ]:
# TODO: Replace the ______ with your own question and run the cell
your_response = llm.invoke("______")
print(your_response)

### 🧪 YOUR TURN — Change the temperature

In [ ]:
# TODO: Create a new pipeline with temperature=0.9
# What changes in the output?

gen_creative = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    max_new_tokens=512,
    temperature=______,  # TODO: set to 0.9
    do_sample=True,
    return_full_text=False
)

creative_llm = HuggingFacePipeline(pipeline=gen_creative)

prompt_text = "Write a one-line tagline for a coffee shop."

print("🧊 Low temp (0.1):", llm.invoke(prompt_text))
print("🔥 High temp (0.9):", creative_llm.invoke(prompt_text))

### Wrap it as a Chat Model

In [ ]:
from langchain_huggingface import ChatHuggingFace

# Wrap as chat model (supports system/human messages)
chat_model = ChatHuggingFace(llm=llm)

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant. Be concise."),
    HumanMessage(content="What is LangChain?"),
]

response = chat_model.invoke(messages)
print(response.content)

### 🧪 YOUR TURN — Change the system persona

In [ ]:
# TODO: Change the SystemMessage to make the model respond
# as a pirate, a poet, or a sports commentator

messages = [
    SystemMessage(content="______"),  # TODO: Write your persona
    HumanMessage(content="Explain what an API is."),
]

response = chat_model.invoke(messages)
print(response.content)

### Create a prompt template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Define a reusable prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a customer support expert. Be concise and professional."),
    ("human", "Classify this support ticket: {ticket_text}")
])

# Test it — see what the formatted prompt looks like
formatted = prompt.format_messages(ticket_text="I was charged twice for my subscription!")
for msg in formatted:
    print(f"[{msg.type}]: {msg.content}")

### 🧪 YOUR TURN — Build your own template

In [ ]:
# TODO: Create a prompt template for a DIFFERENT use case
# Ideas: email writer, tweet generator, code explainer, recipe suggester

my_prompt = ChatPromptTemplate.from_messages([
    ("system", "______"),           # TODO: System instruction
    ("human", "______: {______}")   # TODO: Human message with a variable
])

# Test it
formatted = my_prompt.format_messages(______="______")  # TODO: Fill variable
for msg in formatted:
    print(f"[{msg.type}]: {msg.content}")

###  🧪 YOUR TURN — Multiple variables

In [ ]:
# TODO: Create a template that uses TWO variables
# Example: "Write a {tone} email to {recipient} about ..."

multi_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional email writer."),
    ("human", "Write a {______} email to {______} about a project delay.")  # TODO: Add 2 variables
])

formatted = multi_prompt.format_messages(______="______", ______="______")  # TODO: Fill both
for msg in formatted:
    print(f"[{msg.type}]: {msg.content}")

### Define the Pydantic schema

In [ ]:
from pydantic import BaseModel, Field

class TicketClassification(BaseModel):
    """Schema for classifying customer support tickets."""
    category: str = Field(description="Category: Billing, Technical, Account, or General")
    urgency: str = Field(description="Urgency level: Low, Medium, High, or Critical")
    sentiment: str = Field(description="Customer sentiment: Positive, Neutral, Negative, or Angry")
    summary: str = Field(description="One-line summary of the issue")
    suggested_action: str = Field(description="Recommended next step for the support team")

print("✅ Schema defined!")
print("Fields:", list(TicketClassification.model_fields.keys()))

### 🧪 YOUR TURN — Add a new field

In [ ]:
# TODO: Add a new field to the schema. Ideas:
# - department: str (which team should handle this?)
# - is_escalation: bool (does this need a manager?)
# - estimated_response_time: str (how fast should we respond?)

class TicketClassificationV2(BaseModel):
    """Extended schema with your custom field."""
    category: str = Field(description="Category: Billing, Technical, Account, or General")
    urgency: str = Field(description="Urgency level: Low, Medium, High, or Critical")
    sentiment: str = Field(description="Customer sentiment: Positive, Neutral, Negative, or Angry")
    summary: str = Field(description="One-line summary of the issue")
    suggested_action: str = Field(description="Recommended next step for the support team")
    ______: ______ = Field(description="______")  # TODO: Add your field

print("Fields:", list(TicketClassificationV2.model_fields.keys()))

### Create the output parser

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

# Create a parser from our schema
parser = PydanticOutputParser(pydantic_object=TicketClassification)

# See what instructions the parser generates for the LLM
print(parser.get_format_instructions())

### 🤔 Discussion Question
Look at the format instructions printed above.

1. What format is the LLM being asked to return?
2. Why does the parser need to tell the LLM *how* to format its response?
3. What would happen if we skipped the parser and just used raw LLM text?

### Build the full prompt with parser instructions

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a customer support classifier. "
     "Analyze the support ticket and classify it.\n\n"
     "{format_instructions}"),
    ("human", "{ticket_text}")
])

formatted = prompt.format_messages(
    format_instructions=parser.get_format_instructions(),
    ticket_text="I was charged twice!"
)
print(formatted[0].content[:300], "...")

### Build the chain

In [ ]:
# Build the chain: prompt → model → parser
chain = prompt | chat_model | parser

print("✅ Chain built!")
print("Pipeline: prompt → chat_model → parser")

### 🧪 YOUR TURN — Build the chain yourself

In [ ]:
# TODO: Without looking above, build the chain from scratch
# Fill in the three components in the right order

my_chain = ______ | ______ | ______  # TODO: prompt, chat_model, or parser?

# Test it
test_result = my_chain.invoke({
    "ticket_text": "Your app keeps crashing on my iPhone.",
    "format_instructions": parser.get_format_instructions()
})

print(f"Category: {test_result.category}")
print(f"Urgency:  {test_result.urgency}")

### Run the chain

In [ ]:
result = chain.invoke({
    "ticket_text": "I've been charged twice for my subscription this month and nobody is responding to my emails. I want a refund NOW or I'm canceling.",
    "format_instructions": parser.get_format_instructions()
})

print(f"Category:         {result.category}")
print(f"Urgency:          {result.urgency}")
print(f"Sentiment:        {result.sentiment}")
print(f"Summary:          {result.summary}")
print(f"Suggested Action: {result.suggested_action}")

### See it as JSON

In [ ]:
import json
print(json.dumps(result.model_dump(), indent=2))

### Test on multiple tickets

In [ ]:
test_tickets = [
    "My login isn't working and I have a presentation in 10 minutes!",
    "Hey, just wanted to say your product is fantastic. Keep it up!",
    "How do I change my subscription plan? I can't find the option.",
    "This is the THIRD time my order has been wrong. I want to speak to a manager.",
    "Can you help me integrate your API with my Python project?",
]

print("=" * 60)
for i, ticket in enumerate(test_tickets, 1):
    print(f"\n🎫 Ticket {i}: {ticket}\n")
    try:
        result = chain.invoke({
            "ticket_text": ticket,
            "format_instructions": parser.get_format_instructions()
        })
        print(f"   Category:  {result.category}")
        print(f"   Urgency:   {result.urgency}")
        print(f"   Sentiment: {result.sentiment}")
        print(f"   Summary:   {result.summary}")
        print(f"   Action:    {result.suggested_action}")
    except Exception as e:
        print(f"   ❌ Error: {e}")
    print("-" * 60)

### 🧪 YOUR TURN — Add your own tickets

In [ ]:
# TODO: Add 3 of your own support tickets to this list
my_tickets = [
    "______",  # TODO: Write ticket 1
    "______",  # TODO: Write ticket 2
    "______",  # TODO: Write ticket 3
]

for i, ticket in enumerate(my_tickets, 1):
    print(f"\n🎫 My Ticket {i}: {ticket}\n")
    result = chain.invoke({
        "ticket_text": ticket,
        "format_instructions": parser.get_format_instructions()
    })
    print(json.dumps(result.model_dump(), indent=2))
    print("-" * 60)

## Take Home: 🏆 CHALLENGE — Build a completely different classifier

In [ ]:
# 🏆 CHALLENGE: Build your OWN classifier from scratch!
# Pick one:
#   - Movie review → genre, rating, mood, one-line-summary
#   - Job posting → role_level, department, remote_or_onsite, key_skills
#   - Food review → cuisine, price_range, would_recommend, highlights
#
# Steps:
#   1. Define a Pydantic schema
#   2. Create a parser
#   3. Write a prompt template
#   4. Build the chain
#   5. Test it!

# Step 1: Define your schema
class ______(BaseModel):
    """______"""
    ______: str = Field(description="______")
    ______: str = Field(description="______")
    ______: str = Field(description="______")
    ______: str = Field(description="______")

# Step 2: Create the parser
my_parser = PydanticOutputParser(pydantic_object=______)

# Step 3: Write the prompt
my_prompt = ChatPromptTemplate.from_messages([
    ("system", "______\n\n{format_instructions}"),
    ("human", "{______}")
])

# Step 4: Build the chain
my_chain = ______ | ______ | ______

# Step 5: Test it!
my_result = my_chain.invoke({
    "______": "______",
    "format_instructions": my_parser.get_format_instructions()
})

print(json.dumps(my_result.model_dump(), indent=2))

## 🎯 Recap

### What You Learned

#### 1. Models
You connected to an open-source LLM through Hugging Face's free Inference API using `HuggingFaceEndpoint` and `ChatHuggingFace`. You saw that LangChain is model-agnostic — swap one line and your entire pipeline works with a different model.

#### 2. Prompt Templates
You replaced messy f-strings with `ChatPromptTemplate` — reusable, version-able, and variable-injected. You built templates with single and multiple variables, and experimented with system personas.

#### 3. Output Parsers
You defined a `Pydantic` schema and used `PydanticOutputParser` to force the LLM to return structured JSON instead of raw text. This is what separates a demo from a production app.

#### 4. Chains (LCEL)
You piped everything together with the `|` operator:

```python
chain = prompt | chat_model | parser
```

Three pipes. Three transformations. One line of code.